In [76]:
import pymysql
import pandas as pd
import json
import re

In [77]:
import requests
import datetime
from bs4 import BeautifulSoup

In [78]:
import os

In [79]:
import redis
from xpinyin import Pinyin

In [118]:
con = pymysql.connect(
    host='192.168.10.113',
    user='jiashengjie',
    password='jiashengjie',
    database='gaokao')

## 首页新闻

In [28]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:79.0) Gecko/20100101 Firefox/79.0',
    'Accept-Language': 'zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2',
    'Accept-Encoding': 'gzip, deflate',
}

In [35]:
day_3_ago = str(datetime.date.today() -
                datetime.timedelta(days=3))

In [36]:
class Basic_Spyder():

    def start_scrap(self, part_url_dict, rule_func):
        result = {}
        for part_name, part_url in part_url_dict.items():
            res = requests.get(part_url, headers=headers)
            soup = BeautifulSoup(res.content, 'html.parser')
            item_lst = rule_func(soup)
            result[part_name] = item_lst
        
        # 剔除列表为空的子版块
        result = {key:value for key,value in result.items() if len(value)>1}

        return result

In [39]:
# 教育部
gj_edu_url_dict = {'公告公示': 'http://www.moe.gov.cn/jyb_xxgk/s5743/s5972/',
                   '最新文件': 'http://www.moe.gov.cn/was5/web/search?channelid=239993',
                   '工作动态': 'http://www.moe.gov.cn/jyb_xwfb/gzdt_gzdt/'}

In [72]:
def gj_edu_rule(soup):
    item_lst = []
    base_url = 'http://www.moe.gov.cn/'
    gzdt_base_url = 'http://www.moe.gov.cn/jyb_xwfb/gzdt_gzdt/'
    part = soup.find('h2').get_text()
    if part == '重要文件':
        news_lst = soup.find_all('ul')[6].find_all('li')
    else:
        news_lst = soup.find('ul',id='list').find_all('li')
    
    for item in news_lst:
        item_dict = {}
        post_date = item.find('span').get_text()
        if post_date < day_3_ago:
            break
        title = item.find('a').get_text()
        if part == '重要文件':
            article_url = item.find('a')['href']
        elif part == '工作动态':
            article_url = base_url + item.find('a')['href'].split('./')[-1]
        else:
            article_url = base_url + item.find('a')['href'].split('../')[-1]
        item_dict['post_date'] = post_date
        item_dict['title'] = title
        item_dict['article_url'] = article_url
        item_lst.append(item_dict)
    return item_lst

In [73]:
basic_spyder = Basic_Spyder()

In [74]:

gj_edu_result = basic_spyder.start_scrap(gj_edu_url_dict, gj_edu_rule)

In [75]:
gj_edu_result

{'公告公示': [{'post_date': '2020-08-28',
   'title': '教育部办公厅关于做好2020年全国成人高校招生工作的通知',
   'article_url': 'http://www.moe.gov.cn/srcsite/A15/moe_777/202008/t20200827_480730.html'},
  {'post_date': '2020-08-28',
   'title': '教育部关于同意南昌职业大学变更举办者的批复',
   'article_url': 'http://www.moe.gov.cn/srcsite/A03/s181/202008/t20200827_480681.html'},
  {'post_date': '2020-08-28',
   'title': '教育部关于同意北京邮电大学世纪学院变更举办者之一和办学地址的批复',
   'article_url': 'http://www.moe.gov.cn/srcsite/A03/s181/202008/t20200827_480708.html'},
  {'post_date': '2020-08-28',
   'title': '教育部关于同意四川外国语大学重庆南方翻译学院变更举办者之一的批复',
   'article_url': 'http://www.moe.gov.cn/srcsite/A03/s181/202008/t20200827_480696.html'},
  {'post_date': '2020-08-28',
   'title': '教育部关于同意沈阳城市建设学院变更举办者的批复',
   'article_url': 'http://www.moe.gov.cn/srcsite/A03/s181/202008/t20200827_480716.html'}],
 '工作动态': [{'post_date': '2020-08-28',
   'title': '关于收看2020年“开学第一课”的提示',
   'article_url': 'http://www.moe.gov.cn/s5987/202008/t20200828_481754.html'},
  {'post_date': '202